In [1]:
import pymssql
import re
from collections import OrderedDict
from pandas import Series,DataFrame
from tqdm import tqdm_notebook

import pandas as pd
import copy

import urllib.request #이미지 다운로드
import numpy as np

import pickle

import tensorflow as tf

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


Using TensorFlow backend.


In [2]:
# -- [식품] 대 카테고리 내의 [라면,통조림,즉석식품] 중 카테고리 데이터 셋 추출 
conn = pymssql.connect(server='서버',database = "데이터베이스", user="아이디", password="비밀번호")
query1='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151101%'
OPTION(MAXDOP 1)
'''

query2='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151103%'
OPTION(MAXDOP 1)
'''
query3='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151104%'
OPTION(MAXDOP 1)
'''

query4='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151105%'
OPTION(MAXDOP 1)
'''

query5='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151106%'
OPTION(MAXDOP 1)
'''

query6='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151107%'
OPTION(MAXDOP 1)
'''

query7='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151108%'
OPTION(MAXDOP 1)
'''
query8='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151110%'
OPTION(MAXDOP 1)
'''

query9='''
select top 300000 pl_category, pl_goodsnm, pl_imgurl from ENR_MASTER.ELOC2001.dbo.PRICELIST WITH (nolock)
where pl_category like '151112%'
OPTION(MAXDOP 1)
'''

df1 = pd.read_sql(query1,conn)
df2 = pd.read_sql(query2,conn)
df3 = pd.read_sql(query3,conn)
df4 = pd.read_sql(query4,conn)
df5 = pd.read_sql(query5,conn)
df6 = pd.read_sql(query6,conn)
df7 = pd.read_sql(query7,conn)
df8 = pd.read_sql(query8,conn)
df9 = pd.read_sql(query9,conn)

frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9]

result3 = pd.concat(frames)

In [3]:
#소카테만 남기기
result3.pl_category = result3.pl_category.str[4:6]
# 중복 인덱스 펼쳐주기
result3 = pd.concat([result3],ignore_index=True) 

In [4]:
#문장에서 단어 추출기, konlpy 설치 필요
from konlpy.tag import Okt
okt = Okt()
result3.pl_goodsnm = result3.pl_goodsnm.apply(lambda x: okt.nouns(x)) 

ModuleNotFoundError: No module named 'konlpy'

In [ ]:
# LABEL컬럼 만들기(원핫 백터로 바꾸기 위한 컬럼)
result3['LABEL'] = 0
result3.loc[result3['pl_category']=='01','LABEL'] = 0
result3.loc[result3['pl_category']=='03','LABEL'] = 1
result3.loc[result3['pl_category']=='04','LABEL'] = 2
result3.loc[result3['pl_category']=='05','LABEL'] = 3
result3.loc[result3['pl_category']=='06','LABEL'] = 4
result3.loc[result3['pl_category']=='07','LABEL'] = 5
result3.loc[result3['pl_category']=='08','LABEL'] = 6
result3.loc[result3['pl_category']=='12','LABEL'] = 7

In [ ]:
#정보비율 확인
result3.LABEL.value_counts()

In [ ]:
#정보비율 조정해주기
frames3 = [result3[result3.LABEL == 0][0:4000],result3[result3.LABEL == 1][0:4000],result3[result3.LABEL == 2][0:4000],result3[result3.LABEL == 3][0:4000],result3[result3.LABEL == 4][0:4000],result3[result3.LABEL == 5][0:4000],result3[result3.LABEL == 6][0:4000],result3[result3.LABEL == 7][0:4000]]
result3 = pd.concat(frames3)

In [ ]:
#이미지URL 픽셀화
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.util import img_as_ubyte
from tqdm import tqdm_notebook
for index, row in tqdm_notebook(result3.iterrows()):
    try:
        result3.at[index,'pl_imgurl'] = img_as_ubyte(resize(imread(row['pl_imgurl']),(28,28)))/256
    except:
        result3.at[index,'pl_imgurl'] = None

In [ ]:
#픽셀화 되지못한 None값 가진 row 제거
result3 = result3.dropna()
#인덱스 재정렬 해주기
result3 = pd.concat([result3],ignore_index=True)
result3

In [ ]:
#피클 파일 저장하기
#2:단어분리 3:LABEL 붙임 4:4천개씩만 5:백개씩만 image 전처리6:백개씩 image전처리7:4000개씩 120사이즈 8:84사이즈 9:56사이즈 10: 4000 ~8000 11: ~8000 12: 8000~20000 13: 4000~8000 254사이즈14:20000개 28
with open('shoda4000.pickle','wb') as f:
    pickle.dump(result3, f, pickle.HIGHEST_PROTOCOL)